<a href="https://colab.research.google.com/github/pacificspatial/flateau/blob/main/notebook/sdsc_bootcamp_tokyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Duckdb is already in Colab!
import duckdb

[(42,)]


In [4]:
# simple sql with an in-memory database
duckdb.sql('SELECT 42').show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [6]:
# check relation
r1 = duckdb.sql('SELECT 42 AS i')
duckdb.sql('SELECT i * 2 AS k FROM r1').show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



In [32]:
con = duckdb.connect(database=":memory:", read_only=False, config={"allow_unsigned_extensions": "true"});

In [33]:
con.execute("INSTALL 'httpfs';");

In [34]:
con.execute("INSTALL 'httpfs';");

In [46]:
duckdb.sql('LOAD httpfs')

In [36]:
duckdb.sql('LOAD spatial')

In [47]:
duckdb.sql('SELECT ST_POINT(0,0)')

┌────────────────┐
│ st_point(0, 0) │
│    geometry    │
├────────────────┤
│ POINT (0 0)    │
└────────────────┘

In [49]:
duckdb.sql('select * from duckdb_extensions()')

┌──────────────────┬─────────┬───────────┬──────────────────────┬──────────────────────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │     install_path     │           description            │      aliases      │
│     varchar      │ boolean │  boolean  │       varchar        │             varchar              │     varchar[]     │
├──────────────────┼─────────┼───────────┼──────────────────────┼──────────────────────────────────┼───────────────────┤
│ autocomplete     │ false   │ false     │                      │ Add supports for autocomplete …  │ []                │
│ fts              │ true    │ true      │ (BUILT-IN)           │ Adds support for Full-Text Sea…  │ []                │
│ h3               │ false   │ false     │ /root/.duckdb/exte…  │                                  │ []                │
│ httpfs           │ true    │ true      │ /root/.duckdb/exte…  │ Adds support for reading and w…  │ [http, https, s3] │
│ icu              │ true    │ t

In [39]:
!wget https://github.com/pacificspatial/flateau/raw/main/extensions/h3.duckdb_extension

--2023-09-15 21:56:31--  https://github.com/pacificspatial/flateau/raw/main/extensions/h3.duckdb_extension
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pacificspatial/flateau/main/extensions/h3.duckdb_extension [following]
--2023-09-15 21:56:31--  https://raw.githubusercontent.com/pacificspatial/flateau/main/extensions/h3.duckdb_extension
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433760 (424K) [application/octet-stream]
Saving to: ‘h3.duckdb_extension.1’

h3.duckdb_extension 100%[===================>] 423.59K  --.-KB/s    in 0.02s   

2023-09-15 21:56:31 (16.7 MB/s) - ‘h3.duckdb_ext

In [50]:
con.execute("INSTALL 'h3.duckdb_extension'");

In [51]:
con.execute('LOAD h3;')

IOException: ignored